# Caras lab fiber photometry analysis pipeline
This pipeline is intended to be run after extracting processed signals with our [MatLab pipeline](https://github.com/caraslab/caraslab-fiberphotometry)

Files need to be organized in a specific folder structure or file paths need to be changed

File structure can be found in the Sample dataset folder

## Imports and global plotting parameters
Specific imports can be found within each function

Imports do not need to be changed

Change plotting parameters if desired then rerun this block

In [2]:
%load_ext autoreload
%autoreload 2

from os import remove, makedirs
import warnings
from platform import system
from os.path import sep
from re import split
from datetime import datetime

from multiprocessing import Pool, cpu_count
from glob import glob

from matplotlib.pyplot import rcParams

from helpers.run_FP_pipeline import run_pipeline
from helpers.plot_heatmapsByAnimal import plot_heatmapsByAnimal
from helpers.write_json import write_json

warnings.filterwarnings("ignore")

# Tweak the regex file separator for cross-platform compatibility
if system() == 'Windows':
    REGEX_SEP = sep * 2
else:
    REGEX_SEP = sep

# Set plotting parameters
label_font_size = 11
tick_label_size = 7
legend_font_size = 6
line_thickness = 1

rcParams['figure.dpi'] = 600
rcParams['pdf.fonttype'] = 42
rcParams['ps.fonttype'] = 42
rcParams['font.family'] = 'Arial'
rcParams['font.weight'] = 'regular'
rcParams['axes.labelweight'] = 'regular'

rcParams['font.size'] = label_font_size
rcParams['axes.labelsize'] = label_font_size
rcParams['axes.titlesize'] = label_font_size
rcParams['axes.linewidth'] = line_thickness
rcParams['legend.fontsize'] = legend_font_size
rcParams['xtick.labelsize'] = tick_label_size
rcParams['ytick.labelsize'] = tick_label_size
rcParams['errorbar.capsize'] = label_font_size
rcParams['lines.markersize'] = line_thickness
rcParams['lines.linewidth'] = line_thickness

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Set global paths and variables

Edit this to match your needs. 

This code batch analyzes folders containing recordings from multiple subjects and sessions.

Organize your data in the folder structure illustrated in the Sample dataset

In [18]:
DATA_PATH = r'G:\My Drive\Documents\PycharmProjects\Caraslab_FP_analysis_pipeline\Sample dataset'

# Specify I/O paths and run parameters
SETTINGS_DICT = {
    'EXPERIMENT_TAG': 'GCaMP8s',  # Appends to start of summary files
    
    'EXPERIMENT_TYPE': 'AversiveAM',  # 1IFC or AversiveAM
    
    'SIGNALS_PATH': DATA_PATH + sep + 'Whole session signal',
    'KEYS_PATH': DATA_PATH + sep + 'Key files',
    'OUTPUT_PATH': DATA_PATH + sep + 'Output',
    
    # in seconds; for zscoring and AUC calculation
    'BASELINE_START_FOR_ZSCORE': 0.25,  # Affects plotting and AUC calculation
    'BASELINE_END_FOR_ZSCORE': 0.,  # Affects AUC calculation
    'AUC_WINDOW_START': 0.,  # Affects AUC calculation
    'AUC_WINDOW_END': 4,  # Affects AUC calculation
    'RESPONSE_WINDOW_DURATION': 4,  # Does not affect AUC calculation; only for plotting and signal extraction
     
    # Ignore trials with responses shorter than this (indicative of impulsive behavior for 1IFC task).
    # Keep at 0 for AversiveAM task or if not desired
    'RESPONSE_LATENCY_FILTER': 0.,
    
    # Do you want to analyze responses aligned to trial onset or to response?
    'TRIAL_OR_RESPONSE_ALIGNED': 'trial_aligned',  # 'trial_aligned' OR 'response_aligned'
    
    # Are your response values in milliseconds?
    # This is important if you want to align by response. 
    'MS_LATENCY_VALUES': True,
    
    # Shock is specific to AversiveAM task and only used if you need to recalculate response latencies.
    # Older versions of RPvds did not record latencies properly
    'SHOCK_START_END': [0.95, 1.3],
    
    # Only affects plot shading but keep this in mind when calculating AUC
    'TARGET_SOUND_ONSET': 0.,
    'TARGET_SOUND_OFFSET': 1.,
    
    # For multiprocessing. Defaults to 4/5s of the number of cores
    'NUMBER_OF_CORES': 4 * cpu_count() // 5,
    
    # Only run these [cells/subjects/sessions] ( has to be a list() ) or None to run all
    # You can specify parts of the file name too or a file path with header 'Session'
    'SESSIONS_TO_RUN': None,  
    
    'SESSIONS_TO_EXCLUDE': None,
    
    'CONCAT_SAME_DAY': True,
    
    'DEBUG_RUN': False,  # Turns off multiprocessing for easier debugging
    
    'OVERWRITE_PREVIOUS_CSV': True,  # False: appends to existing firing rate CSV file

    # If None: will be estimated based on diff(Time_s) in the csv file
    'SAMPLING_RATE': None,
    
    # max sampling rate for processing. Downsample if above this
    'DOWNSAMPLE_RATE': None,  # in Hz
    
    'SUBTRACT_405': True,
    
    # These are heatmap parameters for displaying z-scores across all subjects
    'BIN_SIZE': 0.1,  # in seconds
        'TRIALTYPE_DICT': {
            'Hit': [1, 2.5],
            # 'Hit (no shock)': [0, 1.5],
            'False alarm': [1, 2.5],
            'Miss': [1.4, 2.9],
            'Reject': [1.4, 2.9],
            # 'Passive': [0, 1.5],
        },
    
    'SORT_BY_WHICH_TRIALTYPE': 'Hit',
    
    'PLOT_PRETRIAL_DURATION': 0.5,
    
    'PLOT_POSTTRIAL_DURATION': 4,
    
    # You can set this to None and colors will be set automatically using a default colormap (max N=20 subjects for default)
    'SUBJECT_COLORS': None,  
    
    # Below is a switchboard of functions you desire to run from the pipeline
    # If you change your mind later, you can just run the ones you want and the code will add it to existing JSON files
    'PIPELINE_SWITCHBOARD': {
        # Only relevant to AversiveAM task when recorded using older RPvds circuit
        'recalculate_ePsych_responseLatency': False,
        
        # Relevant for both AversiveAM and 1IFC tasks
        'extract_trial_zscores': True,
        'output_sessionData_json': True,  # Happens within extract_trial_zscores, so both have to be true for jsons to be generated
        'plot_trial_zscores': True,
        'plot_AMDepth_zscores': True,
        'plot_heatmaps_by_subject': True
    }
}


## Initial file matching then run pipelines

No need to change anything here

In [20]:
# Log the settings dictionary for reference
write_json(SETTINGS_DICT, SETTINGS_DICT['OUTPUT_PATH'] + sep + 'Aligned signals', 'settings_dict_' + datetime.now().strftime("%m%d%y%H%M%S") + '.json')

# Create directory for session data json files
makedirs(SETTINGS_DICT['OUTPUT_PATH'] + sep + 'JSON files', exist_ok=True)

# Clear older multiprocessing temp files if they exist
process_tempfiles = glob(SETTINGS_DICT['OUTPUT_PATH'] + sep + '*_tempfile_*.csv')
[remove(f) for f in process_tempfiles]

signals_path = glob(SETTINGS_DICT['SIGNALS_PATH'] + sep + '*dff.csv')

if SETTINGS_DICT['SESSIONS_TO_RUN'] is not None:
    signals_path = [path for path in signals_path if any([chosen for chosen in SETTINGS_DICT['SESSIONS_TO_RUN'] if chosen in path])]

if SETTINGS_DICT['SESSIONS_TO_EXCLUDE'] is not None:
    signals_path = [path for path in signals_path if not any([chosen for chosen in SETTINGS_DICT['SESSIONS_TO_EXCLUDE'] if chosen in path])]

if len(signals_path) == 0:
    raise UserWarning('No signal files were found. Please check your paths.')

# Identify sessions from the same subject in a day if desired. Otherwise run session at a time
subj_session_list = [split('_*_', split(REGEX_SEP, path)[-1])[0:2] for path in signals_path]
date_list = [split('-*-', subj_session[1])[3] for subj_session in subj_session_list]
subj_date_list = [(subj_session[0], cur_date) for subj_session, cur_date in zip(subj_session_list, date_list)]

# Generate a list of inputs to be passed to each worker
input_lists = list()
run_list = list()
if SETTINGS_DICT['CONCAT_SAME_DAY']:
    # Identify sessions from the same subject in a day
    subj_session_list = [split('_*_', split(REGEX_SEP, path)[-1])[0:2] for path in signals_path]
    date_list = [split('-*-', subj_session[1])[2] for subj_session in subj_session_list]
    run_list = [(subj_session[0], cur_date) for subj_session, cur_date in zip(subj_session_list, date_list)]
else:
    run_list = signals_path

cur_date_paths_list = list()
for dummy_idx, unique_runID in enumerate(run_list):
    if SETTINGS_DICT['CONCAT_SAME_DAY']:
        (unique_subj, unique_date) = unique_runID
        subj_paths = [path for path in signals_path if unique_subj in path]
        cur_date_paths = [path for path in subj_paths if unique_date in path]
    else:
        cur_date_paths = (unique_runID,)

    # Avoid duplicated runs
    if cur_date_paths in cur_date_paths_list:
        continue
    else:
        cur_date_paths_list.append(cur_date_paths)

for cur_date_paths in cur_date_paths_list:
    if SETTINGS_DICT['DEBUG_RUN']:
        run_pipeline((cur_date_paths, SETTINGS_DICT))
    else:
        input_lists.append((cur_date_paths, SETTINGS_DICT))

if not SETTINGS_DICT['DEBUG_RUN']:
    pool = Pool(SETTINGS_DICT['NUMBER_OF_CORES'])

    # # Feed each worker with all memory paths from one unit
    pool_map_result = pool.map(run_pipeline, input_lists)

    pool.close()

    pool.join()

Extract average trial signals per session and grouping by subject in heatmaps

In [60]:
# Summary heatmaps are not run using multiprocessing because they combine all files
if SETTINGS_DICT['PIPELINE_SWITCHBOARD']['plot_heatmaps_by_subject']:
    rcParams['figure.figsize'] = (4, 3)
    plot_heatmapsByAnimal(SETTINGS_DICT)

Loading data in CSVs...
Uniformizing signal lengths for plotting...
Done!
Adding missing sessions as NaN...
Done!
Organizing and sorting signals for plotting...
Done!
Plotting...
Done!
